# Import Library Dasar

In [16]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

# Cek GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Define compute_metrics di sini agar bisa diakses di mana saja
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    f1_macro = f1_score(labels, preds, average='macro')
    f1_weighted = f1_score(labels, preds, average='weighted')
    return {"accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted}

Device: cpu


# Load Data dan Mapping Label

In [17]:
data_path = "C:/Users/Razy31/Documents/minHackathon/projek-analisis-sentimen-fenomena-vibecoding/data/vibe_coding_auditLabel.csv"

df = pd.read_csv(data_path,sep=';', encoding='utf-8')
df.head()

,video_id,comment_id,parent_id,author,text,like_count,published_at,updated_at,reply_count,tokens,token_len,text_raw,text_trunc,sentiment_pseudo,sentiment_pseudo_id
0,2wwp3dKbGE8,Ugwj1M5bLivVlqF4KkZ4AaABAg,NaN,musfifirah7538,"['bang', 'gimana', 'kalau', 'pemula', 'banget'...",0,2025-11-19T11:44:32Z,2025-11-19T11:44:32Z,0.0,"['bang', 'gimana', 'kalau', 'pemula', 'banget'...",21,bang gimana kalau pemula banget tapi mau belaj...,bang gimana kalau pemula banget tapi mau belaj...,netral,1.0
1,2wwp3dKbGE8,UgzZwHJ7sB8GMMgehFB4AaABAg,NaN,advhe77,"['prinsip2', 'dasar', 'foundamentalnya', 'jgn'...",0,2025-11-18T02:28:17Z,2025-11-18T02:28:17Z,0.0,"['prinsip2', 'dasar', 'foundamentalnya', 'jgn'...",26,prinsip2 dasar foundamentalnya jgn smpe dilupa...,prinsip2 dasar foundamentalnya jgn smpe dilupa...,netral,1.0
2,2wwp3dKbGE8,UgydMEecb1Nh-PRBgq94AaABAg,NaN,zororaka,"['sekedar', 'sharing', 'aja', 'pernah', 'disku...",0,2025-11-17T05:25:58Z,2025-11-17T05:29:49Z,0.0,"['sekedar', 'sharing', 'aja', 'pernah', 'disku...",45,sekedar sharing aja pernah diskusi tech lead b...,sekedar sharing aja pernah diskusi tech lead b...,netral,1.0
3,2wwp3dKbGE8,UgxoF9XGiQczCku-ZqJ4AaABAg,NaN,reafterstudio,"['bukan', 'programer', 'tapi', 'butuh', 'websi...",0,2025-11-14T23:31:39Z,2025-11-14T23:31:39Z,0.0,"['bukan', 'programer', 'tapi', 'butuh', 'websi...",70,bukan programer tapi butuh website sederhana u...,bukan programer tapi butuh website sederhana u...,positif,2.0
4,2wwp3dKbGE8,UgzAbfd4HeGD1h1UxPJ4AaABAg,NaN,kurabasakurata2575,"['biasa', 'ngehandle', 'pekerjaan', 'sendiri',...",0,2025-11-13T00:32:17Z,2025-11-13T00:32:17Z,0.0,"['biasa', 'ngehandle', 'pekerjaan', 'sendiri',...",51,biasa ngehandle pekerjaan sendiri punya gaya k...,biasa ngehandle pekerjaan sendiri punya gaya k...,netral,1.0


In [18]:
df['sentiment_pseudo'].value_counts()

sentiment_pseudo
netral     630
negatif    134
positif    115
Name: count, dtype: int64

In [19]:
# Pastikan kolom yang diperlukan sudah ada
assert 'text_raw' in df.columns
assert 'sentiment_pseudo' in df.columns

df['text_raw'] = (
    df['text_raw'].astype(str).fillna("").
    str.replace('\n', ' ',regex=False).
    str.strip()
)

label_map = {
    "negatif": 0,
    "netral": 1,
    "positif": 2
}

df['label'] = df['sentiment_pseudo'].str.lower().map(label_map)
df['label'].value_counts().sort_index()


label
0    134
1    630
2    115
Name: count, dtype: int64

# Helper: Split, Oversampling, Tokenization

In [20]:
# Stratified split train dan val
def make_splits(df, test_size=0.2, seed=42):
    X_train, X_val, y_train, y_val = train_test_split(
        df['text_raw'].tolist(),
        df['label'].tolist(),
        test_size=test_size,
        random_state=seed,
        stratify=df['label'].tolist(),
    )
    train_df = pd.DataFrame({'text_raw': X_train, 'label': y_train})
    val_df = pd.DataFrame({'text_raw': X_val, 'label': y_val})
    return train_df, val_df

train_df_base, val_df_base = make_splits(df)
print("Train label dist:",train_df_base['label'].value_counts().sort_index().to_dict())
print("Val label dist:",val_df_base['label'].value_counts().sort_index().to_dict())

Train label dist: {0: 107, 1: 504, 2: 92}
Val label dist: {0: 27, 1: 126, 2: 23}


In [21]:
# Oversampling minoritas di train set
def oversampling_minority(df, label_col='label', target='label', target_per_class=250, seed=42):
    rng = np.random.default_rng(seed)
    dfs = []
    
    for lbl, group in df.groupby(label_col):
        if len(group) >= target_per_class:
            dfs.append(group)
        else:
            needed = target_per_class - len(group)
            extra_idx = rng.choice(group.index, size=needed, replace=True)
            extra_samples = group.loc[extra_idx]
            dfs.append(pd.concat([group, extra_samples], axis=0))
    df_os = pd.concat(dfs, axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    return df_os


# Tokenization Helper
model_name = 'indolem/indobert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(
        batch['text_raw'],
        truncation=True,
        padding='max_length',
        max_length=256,
    )

# WeightTrainer & Metrics 

In [22]:
class WeightedTrainer(Trainer):
    """
    Trainer HF + CrossEntropyLoss dengan class weights
    """
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        if self.class_weights is not None:
            cw = self.class_weights.to(logits.device)
            loss_fct = nn.CrossEntropyLoss(weight=cw)
            
        else:
            loss_fct = nn.CrossEntropyLoss()
        
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Buat `run_experiment` untuk 1 strategi

In [ ]:
def run_experiment(
    strategy_name: str,
    base_train_df: pd.DataFrame,
    base_val_df: pd.DataFrame,
    oversample: bool,
    use_class_weight: bool,
    epochs: int = 5,
    batch_size: int = 8,
    lr: float=2e-5,
    target_per_class: int = 250,
    output_root: str = "../models/experimentsIndobert"
):
    """
    Jalankan satu eksperimen:
    - oversampling train (opsional)
    - class_weights (opsional)
    - fine-tune IndoBERT
    - evaluate di val
    """
    print(f"\n===== Experiment: {strategy_name} =====")
    
    # 1) siapkan train_df sesuai strategi
    if oversample:
        train_df = oversampling_minority(
            base_train_df, target_per_class=target_per_class)
        print("Train label dist (oversampled):", train_df['label'].value_counts().sort_index().to_dict())
    else:
        train_df = base_train_df.copy()
        print("Train Label dist (base):", train_df['label'].value_counts().sort_index().to_dict())
        
        
    val_df = base_val_df.copy()
    print("Val label dist:", val_df['label'].value_counts().sort_index().to_dict())
    
    # 2) Hitung class_weights kalau dipakai
    class_weights_tensor = None
    if use_class_weight:
        label_counts = train_df['label'].value_counts().sort_index()
        counts = label_counts.values.astype(float)
        inv_freq = 1.0 / counts
        class_weights = inv_freq / inv_freq.sum() * len(counts)
        class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
        print("Class Weights (0,1,2):", class_weights_tensor.tolist())
    else:
        print("Class Weigths: Not Used")
        
    # 3) Siapkan dataset HF
    train_dfs = Dataset.from_pandas(train_df.reset_index(drop=True))
    val_dfs = Dataset.from_pandas(val_df.reset_index(drop=True))
    
    train_dfs = train_dfs.map(tokenize_function, batched=True)
    val_dfs = val_dfs.map(tokenize_function, batched=True)
    
    train_dfs = train_dfs.remove_columns(['text_raw'])
    val_dfs = val_dfs.remove_columns(['text_raw'])
    
    train_dfs = train_dfs.with_format("torch")
    val_dfs = val_dfs.with_format("torch")
    
    # 4) Load model baru tiap eksperimen (biar fair)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,
    ).to(device)
    
    # 5) TrainingArguments minimal
    exp_output_dir = os.path.join(output_root, strategy_name)
    os.makedirs(exp_output_dir, exist_ok=True)
    
    training_args = TrainingArguments(
        output_dir=exp_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=lr,
        logging_steps=50,
        save_total_limit=1,
        weight_decay=0.01,
    )
    
    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dfs,
        eval_dataset=val_dfs,
        compute_metrics=compute_metrics,
        class_weights=class_weights_tensor,
    )
    
    # 6) Train
    train_result = trainer.train()
    print("Train result:", train_result.metrics)
    
    # 7) Evaluate
    eval_result = trainer.evaluate()
    print("Eval result:", eval_result)
    
    
    # 8) Prediksi detail untuk classfication report
    pred_output = trainer.predict(val_dfs)
    y_true = pred_output.label_ids
    y_pred = np.argmax(pred_output.predictions, axis=1)
    
    cls_report = classification_report(
        y_true,
        y_pred,
        target_names=['negatif', 'netral', 'positif'],
        digits=3,
        output_dict=True, #disimpan dalam bentuk dictionary
    )
    
    # 9) save model & tokenizer (optional: hanya yang terbaik nanti)
    trainer.save_model(exp_output_dir)
    tokenizer.save_pretrained(exp_output_dir)
    
    return{
                "name": strategy_name,
        "oversample": oversample,
        "use_class_weight": use_class_weight,
        "epochs": epochs,
        "batch_size": batch_size,
        "lr": lr,
        "target_per_class": target_per_class,
        "eval_metrics": eval_result,
        "cls_report": cls_report,
        "output_dir": exp_output_dir,
    }

# Looping semua strategi & ringkasan hasil

In [24]:
strategies = [
    {"name": "baseline",              "oversample": False, "use_class_weight": False},
    {"name": "class_weight_only",     "oversample": False, "use_class_weight": True},
    {"name": "oversample_only",       "oversample": True,  "use_class_weight": False},
    {"name": "oversample_and_weight", "oversample": True,  "use_class_weight": True},
]

experiments = []
for cfg in strategies:
    result = run_experiment(
        strategy_name=cfg["name"],
        base_train_df=train_df_base,
        base_val_df=val_df_base,
        oversample=cfg["oversample"],
        use_class_weight=cfg["use_class_weight"],
        epochs=3,           # bisa kamu ubah global di sini
        batch_size=8,
        lr=2e-5,
        target_per_class=250,
        output_root="models/experiments_indobert",
    )
    experiments.append(result)


===== Experiment: baseline =====
Train Label dist (base): {0: 107, 1: 504, 2: 92}
Val label dist: {0: 27, 1: 126, 2: 23}
Class Weigths: Not Used


Map:   0%|          | 0/703 [00:00<?, ? examples/s]

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.808700
100,0.677400
150,0.677400
200,0.632000
250,0.518100


...

Step,Training Loss
50,1.064500
100,0.953500
150,0.926300
200,0.818600
250,0.730500
300,0.570500
350,0.554100


Train result: {'train_runtime': 2469.7946, 'train_samples_per_second': 1.22, 'train_steps_per_second': 0.153, 'total_flos': 396248807098368.0, 'train_loss': 0.776834593878852, 'epoch': 3.0}


c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Eval result: {'eval_loss': 0.9110493063926697, 'eval_accuracy': 0.5909090909090909, 'eval_f1_macro': 0.5163015708187048, 'eval_f1_weighted': 0.6262060131803122, 'eval_runtime': 33.7217, 'eval_samples_per_second': 5.219, 'eval_steps_per_second': 0.652, 'epoch': 3.0}


c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


# Table Perbandingan & pilih yang terbaik

In [25]:
rows = []
for exp in experiments:
    m = exp["eval_metrics"]
    rows.append({
        "name": exp["name"],
        "oversample": exp["oversample"],
        "class_weight": exp["use_class_weight"],
        "epochs": exp["epochs"],
        "batch_size": exp["batch_size"],
        "lr": exp["lr"],
        "eval_loss": m.get("eval_loss", None),
        "accuracy": m.get("eval_accuracy", None),
        "f1_macro": m.get("eval_f1_macro", None),
        "f1_weighted": m.get("eval_f1_weighted", None),
        "output_dir": exp["output_dir"],
    })

summary_df = pd.DataFrame(rows).sort_values(by="f1_macro", ascending=False)
summary_df

,name,oversample,class_weight,epochs,batch_size,lr,eval_loss,accuracy,f1_macro,f1_weighted,output_dir
2,oversample_only,True,False,3,8,0.00002,0.693879,0.676136,0.563903,0.692999,models/experiments_indobert\oversample_only
1,class_weight_only,False,True,3,8,0.00002,0.900191,0.613636,0.522366,0.640167,models/experiments_indobert\class_weight_only
3,oversample_and_weight,True,True,3,8,0.00002,0.911049,0.590909,0.516302,0.626206,models/experiments_indobert\oversample_and_weight
0,baseline,False,False,3,8,0.00002,0.788919,0.693182,0.407843,0.641845,models/experiments_indobert\baseline


In [26]:
# Pilih eksperimen terbaik berdasarkan f1_macro
best_exp = max(experiments, key=lambda e: e["eval_metrics"].get("eval_f1_macro", 0))
best_exp["name"], best_exp["eval_metrics"], best_exp["output_dir"]

('oversample_only',
 {'eval_loss': 0.6938790678977966,
  'eval_accuracy': 0.6761363636363636,
  'eval_f1_macro': 0.563903494536406,
  'eval_f1_weighted': 0.6929990262901655,
  'eval_runtime': 32.7647,
  'eval_samples_per_second': 5.372,
  'eval_steps_per_second': 0.671,
  'epoch': 3.0},
 'models/experiments_indobert\\oversample_only')

# Fine Tune untuk `oversample_only`

In [28]:
def make_tokenize_fn(max_length):
    def tokenize_batch(batch):
        return tokenizer(
            batch['text_raw'],
            truncation=True,
            padding='max_length',
            max_length=max_length,
        )
    return tokenize_batch

In [29]:
def run_experiment(
    strategy_name: str,
    base_train_df: pd.DataFrame,
    base_val_df: pd.DataFrame,
    epochs: int =3,
    batch_size: int=8,
    lr: float=2e-5,
    target_per_class: int =250,
    max_length: int =256,
    output_root: str ="../models/fineTuneIndobert",
):
    """
    Satu ekperimen dengan:
    - Oversampling minor class (selalu True disini)
    - tanpa class_weight (oversample_only)
    - Hyperparam bisa di-set: epochs, batch_size, lr, target_per_class, max_length
    """
    print(f"\n===== EXP: {strategy_name} =====")
    print(f"epochs={epochs}, lr={lr}, target_per_class={target_per_class}, max_len={max_length}, batch={batch_size}")

    # 1) Oversample train
    train_df = oversampling_minority(base_train_df, target_per_class=target_per_class)
    val_df   = base_val_df.copy()

    print("Train label dist (oversampled):", train_df["label"].value_counts().sort_index().to_dict())
    print("Val   label dist:", val_df["label"].value_counts().sort_index().to_dict())

    # 2) Dataset HF + tokenization sesuai max_length
    tok_fn = make_tokenize_fn(max_length=max_length)

    train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
    val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

    train_ds = train_ds.map(tok_fn, batched=True)
    val_ds   = val_ds.map(tok_fn, batched=True)

    train_ds = train_ds.remove_columns(["text_raw"])
    val_ds   = val_ds.remove_columns(["text_raw"])

    train_ds = train_ds.with_format("torch")
    val_ds   = val_ds.with_format("torch")

    # 3) Model fresh
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=3,
    ).to(device)

    # 4) TrainingArguments minimal
    exp_output_dir = os.path.join(output_root, strategy_name)
    os.makedirs(exp_output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=exp_output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=lr,
        logging_steps=50,
        save_total_limit=1,
        weight_decay=0.01,
    )

    trainer = WeightedTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
        class_weights=None,  # oversample_only => TANPA class_weight
    )

    # 5) Train + eval
    train_result = trainer.train()
    print("Train metrics:", train_result.metrics)

    eval_metrics = trainer.evaluate()
    print("Eval metrics:", eval_metrics)

    # 6) Classification report (biar bisa dicek detail kalau perlu)
    pred_output = trainer.predict(val_ds)
    y_true = pred_output.label_ids
    y_pred = np.argmax(pred_output.predictions, axis=1)

    cls_report = classification_report(
        y_true,
        y_pred,
        target_names=["negatif", "netral", "positif"],
        digits=3,
        output_dict=True,
    )

    # 7) Save model & tokenizer
    trainer.save_model(exp_output_dir)
    tokenizer.save_pretrained(exp_output_dir)

    return {
        "name": strategy_name,
        "epochs": epochs,
        "batch_size": batch_size,
        "lr": lr,
        "target_per_class": target_per_class,
        "max_length": max_length,
        "eval_metrics": eval_metrics,
        "cls_report": cls_report,
        "output_dir": exp_output_dir,
    }

In [31]:
search_configs = [
    {"epochs": 3, "lr": 2e-5, "target_per_class": 250, "max_length": 256},
    {"epochs": 4, "lr": 2e-5, "target_per_class": 250, "max_length": 256},
    {"epochs": 3, "lr": 3e-5, "target_per_class": 250, "max_length": 256},
    {"epochs": 4, "lr": 3e-5, "target_per_class": 250, "max_length": 256},
    {"epochs": 3, "lr": 2e-5, "target_per_class": 300, "max_length": 256},
    {"epochs": 3, "lr": 2e-5, "target_per_class": 250, "max_length": 128},
]

experiments_tuned = []

for i, cfg in enumerate(search_configs):
    name = f"oversample_only_e{cfg['epochs']}_lr{cfg['lr']}_tar{cfg['target_per_class']}_ml{cfg['max_length']}"
    result = run_experiment(
        strategy_name=name,
        base_train_df=train_df_base,
        base_val_df=val_df_base,
        epochs=cfg["epochs"],
        batch_size=8,
        lr=cfg["lr"],
        target_per_class=cfg["target_per_class"],
        max_length=cfg["max_length"],
        output_root="../models/fineTuneIndobert",
    )
    experiments_tuned.append(result)

rows = []
for exp in experiments_tuned:
    m = exp["eval_metrics"]
    rows.append({
        "name": exp["name"],
        "epochs": exp["epochs"],
        "batch_size": exp["batch_size"],
        "lr": exp["lr"],
        "target_per_class": exp["target_per_class"],
        "max_length": exp["max_length"],
        "eval_loss": m.get("eval_loss", None),
        "accuracy": m.get("eval_accuracy", None),
        "f1_macro": m.get("eval_f1_macro", None),
        "f1_weighted": m.get("eval_f1_weighted", None),
        "output_dir": exp["output_dir"],
    })

tuned_summary_df = pd.DataFrame(rows).sort_values(by="f1_macro", ascending=False)
tuned_summary_df


===== EXP: oversample_only_e3_lr2e-05_tar250_ml256 =====
epochs=3, lr=2e-05, target_per_class=250, max_len=256, batch=8
Train label dist (oversampled): {0: 250, 1: 504, 2: 250}
Val   label dist: {0: 27, 1: 126, 2: 23}


Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,1.012800
100,0.841800
150,0.761900
200,0.602000
250,0.528100
300,0.382500
350,0.403600


...

Train metrics: {'train_runtime': 1172.228, 'train_samples_per_second': 2.569, 'train_steps_per_second': 0.322, 'total_flos': 198124403549184.0, 'train_loss': 0.7245212131076388, 'epoch': 3.0}


c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Eval metrics: {'eval_loss': 0.8796199560165405, 'eval_accuracy': 0.6136363636363636, 'eval_f1_macro': 0.5074818955415971, 'eval_f1_weighted': 0.6433426060291733, 'eval_runtime': 20.6736, 'eval_samples_per_second': 8.513, 'eval_steps_per_second': 1.064, 'epoch': 3.0}


c:\Users\Razy31\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


,name,epochs,batch_size,lr,target_per_class,max_length,eval_loss,accuracy,f1_macro,f1_weighted,output_dir
1,oversample_only_e4_lr2e-05_tar250_ml256,4,8,0.00002,250,256,0.887943,0.687500,0.587276,0.705528,../models/fineTuneIndobert\oversample_only_e4_...
0,oversample_only_e3_lr2e-05_tar250_ml256,3,8,0.00002,250,256,0.776114,0.670455,0.587142,0.690631,../models/fineTuneIndobert\oversample_only_e3_...
2,oversample_only_e3_lr3e-05_tar250_ml256,3,8,0.00003,250,256,0.803482,0.664773,0.566556,0.686579,../models/fineTuneIndobert\oversample_only_e3_...
4,oversample_only_e3_lr2e-05_tar300_ml256,3,8,0.00002,300,256,0.770864,0.636364,0.559821,0.661969,../models/fineTuneIndobert\oversample_only_e3_...
5,oversample_only_e3_lr2e-05_tar250_ml128,3,8,0.00002,250,128,0.879620,0.613636,0.507482,0.643343,../models/fineTuneIndobert\oversample_only_e3_...
3,oversample_only_e4_lr3e-05_tar250_ml256,4,8,0.00003,250,256,1.004329,0.619318,0.452900,0.631476,../models/fineTuneIndobert\oversample_only_e4_...


In [32]:
best_tuned = max(experiments_tuned, key=lambda e: e["eval_metrics"].get("eval_f1_macro", 0))
best_tuned["name"], best_tuned["eval_metrics"], best_tuned["output_dir"]

('oversample_only_e4_lr2e-05_tar250_ml256',
 {'eval_loss': 0.8879430294036865,
  'eval_accuracy': 0.6875,
  'eval_f1_macro': 0.5872758535986243,
  'eval_f1_weighted': 0.7055277349768875,
  'eval_runtime': 32.0165,
  'eval_samples_per_second': 5.497,
  'eval_steps_per_second': 0.687,
  'epoch': 4.0},
 '../models/fineTuneIndobert\\oversample_only_e4_lr2e-05_tar250_ml256')